In [3]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as onp
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")


In [ ]:


def amr_abm(agents_state, movement, ward2size, parameters):
    """_summary_

    Args:
        agents_state : agent state. {0: susceptible, 1: colonized}.                 Size: (n_patients)
        movement     : pd.Dataframe with patient locations and culture information.
        parameters   : dictionary of parameters, contains importation rate (gamma), nosocomial transmission rate (beta),
                        effective sensitivity (ro), and decolonization rate (alpha)
    """

    γ   = parameters["gamma"] # importation rate.
    β   = parameters["beta"]  # nosocomial transmission rate.
    ρ   = parameters["ro"]    # effective sensitivity.
    α   = parameters["alpha"] # decolonization rate
    Nw  = parameters["Nw"]    # number of wards

    # active colonizations
    active_colonizations      = onp.where(agents_state==1)[0]
    decolonized               = active_colonizations[onp.random.random(size=(active_colonizations[0].shape)) < α]
    agents_state[decolonized] = 0

    # Import patients.
    new_patients = movement[movement["first_day"]==1]["mrn_id"].values
    if new_patients.shape[0]>0:
        # P(S2C) = \gamma - Probability of colonization given importation.
        agents_state[new_patients]   = onp.random.random(size=(new_patients[0].shape)) < γ

    # compute force of infection for each ward.
    for i, ward_id in enumerate( movement["ward_id"].unique()):
        patients_ward = movement[movement["ward_id"]==ward_id]["mrn_id"].values

        # λ_i = β  * C / N  - Force of infection for ward i.
        λ_i = β * onp.sum(agents_state[patients_ward]==1) / ward2size[ward_id]

        # P(C2S)_i = λ_i, we add the state but if already colonized the state would be 2 so we clip it to 1.
        agents_state[patients_ward] = onp.minimum(agents_state[patients_ward] + onp.random.random(size=(patients_ward.shape)) < λ_i, 1)


    return agents_state
